# core

> a library that wraps jsonschema's `validate()` to validate a configuration dict (`.env` in particular)

In [ ]:
#| default_exp core

In [ ]:
#| export
import json
import logging
import os
import os.path
from typing import Union

import dotenv
import jsonschema
from fsspec.spec import AbstractFileSystem
from fsspec.implementations.local import LocalFileSystem
from fsspec.implementations.dirfs import DirFileSystem
from jsonschema import ValidationError, validate
from io import StringIO

logger = logging.getLogger(__name__)

In [ ]:
#| hide

# dev/testing deps
from unittest.mock import patch

from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#| export
def coerce_primitive_values(json_schema: dict, data: dict) -> dict:
    """
    Given a JSON schema dictionary, return a dictionary where the values that have
    primitive types ('string', 'integer', 'number', 'boolean') 
    as described in the schema are converted to the corresponding types from `str`.
   
    Args:
        - `json_schema` (dict): JSON schema used for extraction. The function expects but does not validate this 
                                to be a JSON schema.
        - `data` (dict): The data (e.g., dotenv, os.environ) to extract from.

    Returns:
        dict: A coerced dictionary with values converted to the corresponding primitive types.
    """
    if not isinstance(data, dict):
        return data
    out = data.copy()
    # use the json schema to convert types on known properties
    for property_name, property_schema in json_schema['properties'].items():
        property_type = property_schema.get('type')
        property_value = data.get(property_name)
        if property_value is None:
            continue
        # property_value should be a string at this point
        try:
            if property_type == 'integer':
                out[property_name] = int(data[property_name])
            elif property_type == 'number':
                out[property_name] = float(data[property_name])
            elif property_type == 'boolean':
                parsed_boolean = None
                if property_value.lower() in (
                    'true', 'yes', 'y', '1', 'on',
                ):
                    parsed_boolean = True
                elif property_value.lower() in (
                    'false', 'no', 'n', '0', 'off',
                ):
                    parsed_boolean = False
                out[property_name] = parsed_boolean
        except:
            # leave any validation error descriptions to json schema
            continue
    return out

In [ ]:
#| hide
# everything is correct
test_eq(coerce_primitive_values({
    'type': 'object',
    'properties': {
        'STRING': { 'type': 'string' },
        'NUMBER': { 'type': 'number' },
        'INTEGER': { 'type': 'number' },
        'BOOLEAN': { 'type': 'boolean' },
    },
}, {
    'STRING': 'asdf',
    'NUMBER': '1232529.56',
    'INTEGER': '98758585858232',
    'BOOLEAN': 'TRUE',
}), {
    'STRING': 'asdf',
    'NUMBER': 1232529.56,
    'INTEGER': 98758585858232,
    'BOOLEAN': True,
})

In [ ]:
#|hide
# undeclared types do not get converted
test_eq(coerce_primitive_values({
    'type': 'object',
    'properties': {
        'STRING': { 'type': 'string' },
        'SOMETHING_ELSE': {},
    },
}, {
    'NUMBER': '1232529.56',
    'INTEGER': '98758585858232',
    'BOOLEAN': 'TRUE',
    'SOMETHING_ELSE': {'a': 1, 'b': ['c', 3]},
}), {
    'NUMBER': '1232529.56',
    'INTEGER': '98758585858232',
    'BOOLEAN': 'TRUE',
    'SOMETHING_ELSE': {'a': 1, 'b': ['c', 3]},
})

In [ ]:
#| export
def extract_declared_items(json_schema: dict, data: dict) -> dict:
    """
    Given a JSON schema dict, return a dict following specified rules:
    
    1. All keys not declared in the schema are removed.
    2. All keys declared in the schema are present. If a key declared in the schema with a default 
       is not present in the original data, it's added with the 'default' value.
    
    Args:
        - `json_schema` (dict): JSON schema used for extraction. The function expects but does not validate this 
                                to be a JSON schema.
        - `data` (dict): The data (for example, dotenv, os.environ) to extract from.

    Returns:
        dict: A dictionary that has been processed according to the rules specified above.
    """
    properties = json_schema['properties']
    out = {key: value for (key, value) in data.items() if key in properties}
    for required_property, property_schema in properties.items():
        if required_property not in out and 'default' in property_schema:
            out[required_property] = property_schema['default']
    return out

In [ ]:
#| hide
test_eq(extract_declared_items({
    'type': 'object',
    'properties': {
        'STRING': { 'type': 'string' },
        'SOMETHING_ELSE': {},
        'HAS_DEFAULT': { 'type': 'boolean', 'default': 'NO COERCION!' },
    },
}, {
    'NUMBER': '1232529.56',
    'INTEGER': '98758585858232',
    'BOOLEAN': 'TRUE',
    'SOMETHING_ELSE': {'a': 1, 'b': ['c', 3]},
}), {
    'SOMETHING_ELSE': {'a': 1, 'b': ['c', 3]},
    'HAS_DEFAULT': 'NO COERCION!',
})

In [ ]:
#| export
class ConfigValidatorException(Exception):
    
    def __init__(self, errors):
        super().__init__('''config failed to validate against JSON schema\n{}'''.format(
            '\n'.join(f'{error.json_path}: {error.message}' for error in errors)
        ))
        self.errors = errors


class ConfigValidator(object):

    # this is a risky value: it gets reused across instances;
    # the idea is to maybe set it once and use it multiple times.
    # but in testing this smells bad
    DEFAULT_STORAGE_DRIVER: AbstractFileSystem = None  # defaults to DirFileSystem
    
    CONFIG_VALIDATOR_JSON_SCHEMA_ENVVAR_NAME = 'CONFIG_VALIDATOR_JSON_SCHEMA'
    
    @classmethod
    def get_default_storage_driver(cls):
        if cls.DEFAULT_STORAGE_DRIVER is None:
            cls.DEFAULT_STORAGE_DRIVER = DirFileSystem(os.getcwd())
        return cls.DEFAULT_STORAGE_DRIVER

    
    @classmethod
    def _get_maybe_abspath_driver(cls, maybe_abspath: str):
        if os.path.isabs(maybe_abspath):  # special case
            return DirFileSystem('/')
        else:
            return cls.get_default_storage_driver()
        
    @classmethod
    def load_json(cls, json_source: Union[str, dict]=None, storage_driver: AbstractFileSystem = None) -> dict:
        """
        Convenience method to return a dictionary from either a file path or an already-loaded dictionary.

        Args:
            - `json_source` (Union[str, dict], optional): The JSON source to load.
                                This can be a file path (str) 
                                or an already loaded dictionary (dict). 
            - `storage_driver` (AbstractFileSystem, optional): An instance of the storage driver used to
                                load the JSON file. If not provided, DirFileSystem from the current
                                working dir is used.

        Returns:
            dict: A dictionary that was loaded from the provided `json_source`.
        """
        if isinstance(json_source, dict):
            return json_source
        
        if storage_driver is None:
            storage_driver = cls._get_maybe_abspath_driver(json_source)
        with storage_driver.open(json_source) as ifile:
            return json.load(ifile)

    @classmethod
    def get_default_json_schema(cls, storage_driver: AbstractFileSystem = None) -> dict:
        if cls.CONFIG_VALIDATOR_JSON_SCHEMA_ENVVAR_NAME in os.environ:
            expected_json_schema_path = \
                os.environ[cls.CONFIG_VALIDATOR_JSON_SCHEMA_ENVVAR_NAME]
            return cls.load_json(expected_json_schema_path, storage_driver)
        return None

    def __init__(self, json_schema: Union[str, dict]=None, storage_driver: AbstractFileSystem=None):
        """
        Initialize the instance with a JSON schema and a storage driver.

        Args:
            - `json_schema` (Union[str, dict], optional): A string path to a JSON schema file, or a schema in dictionary form. 
                                                          If no value is provided, it will fall back to looking for an environment 
                                                          variable corresponding to the class variable 
                                                          `CONFIG_VALIDATOR_JSON_SCHEMA_ENVVAR_NAME` to find a JSON schema file.
            - `storage_driver` (AbstractFileSystem, optional): The storage driver to use. If no value is provided, 
                                                          `self.__class__.DEFAULT_STORAGE_DRIVER` is used.

        Raises:
            Exception: An exception is raised if no valid JSON schema is provided or found.
        """
        if isinstance(json_schema, (str, dict)):
            self._json_schema = self.__class__.load_json(json_schema, storage_driver=storage_driver)
        elif (default_schema := self.__class__.get_default_json_schema(storage_driver=storage_driver)):
            self._json_schema = default_schema
        else:
            raise Exception(
                'did not receive or find a JSON schema (after falling back to os.environ["{}"])'.format(
                    self.__class__.CONFIG_VALIDATOR_JSON_SCHEMA_ENVVAR_NAME,
                ))

    def load_config(self, config: dict):
        extracted_config = extract_declared_items(self._json_schema, config)
        coerced_config = coerce_primitive_values(self._json_schema, extracted_config)
        validator = jsonschema.Draft4Validator(self._json_schema)
        errors = list(validator.iter_errors(coerced_config))
        if errors:
            raise ConfigValidatorException(errors)
        return coerced_config
    
    @classmethod
    def load_validated_config(cls, json_schema: Union[str, dict], config: dict, **kwargs):
        return cls(json_schema, **kwargs).load_config(config)

    @classmethod
    def load_validated_environment(cls, json_schema: Union[str, dict]=None, **kwargs):
        return cls.load_validated_config(json_schema, dict(os.environ), **kwargs)
        
    @classmethod
    def load_dotenv(cls,
                    json_schema: Union[str, dict]=None,
                    dotenv_path: str=None,
                    storage_driver: AbstractFileSystem=None,
                    override: bool=False,
                   ):
        """
        Loads environment variables from a .env file or JSON schema defaults, where applicable, into `os.environ`.

        Args:
            - `json_schema` (Union[str, dict], optional):
                                            A JSON schema used for extraction. It can be a string path to 
                                            a JSON schema file or a dictionary. If not provided, another method 
                                            (such as an environment variable or default schema) is used.
            - `dotenv_path` (str, optional): Path to the .env file to load the variables from.
                                             If not provided, loads an empty dict to start.
            - `storage_driver` (AbstractFileSystem, optional): The storage driver to use for loading files.
                                             If not given, ".env" will be attempted from the current working
                                             directory;  if that does not exist, an empty dict will be used.
            - `override` (bool, optional): If True, variables from the .env file or schema default override existing
                                           `os.environ` variables.
        """

        # WARN this sidesteps storage_driver!
        # it will cause breakage if storage_driver != DirFileSystem AND `.env` exists in PWD
        if dotenv_path is None:
            maybe_dotenv_path = dotenv.find_dotenv()  # '' if not exist; else abspath
            if maybe_dotenv_path:
                logger.debug(f'using detected dotenv path; {maybe_dotenv_path}')
                dotenv_path = maybe_dotenv_path
        
        config = None
        
        if dotenv_path:
            dotenv_storage_driver = storage_driver or cls._get_maybe_abspath_driver(dotenv_path)
            with dotenv_storage_driver.open(dotenv_path) as ifile:
                config = dotenv.dotenv_values(stream=StringIO(ifile.read().decode('utf-8')))
                
        if config is None:
            dotenv_storage_driver = storage_driver or cls.get_default_storage_driver()
            if dotenv_storage_driver.exists('.env'):  # unlike dotenv.find_dotenv, stay relative!
                with dotenv_storage_driver.open('.env') as ifile:
                    config = dotenv.dotenv_values(stream=StringIO(ifile.read().decode('utf-8')))
        
        if config is None:
            config = {}
        
        if json_schema is not None:
            json_schema_dict = cls.load_json(json_schema, storage_driver=storage_driver) or {}
        else: 
            json_schema_dict = cls.get_default_json_schema(storage_driver)
            
        for key in json_schema_dict.get('properties', []):
            if key not in os.environ:
                continue
            if key in config and config[key] != os.environ[key]:
                logger.debug(f'os.environ key "{key}" overriding value present in {dotenv_path}')
            config[key] = os.environ[key]
        validated_config = cls.load_validated_config(
            json_schema or cls.get_default_json_schema(storage_driver=storage_driver),
            config, storage_driver=storage_driver)
        
        if override:
            for key, value in validated_config.items():
                if key in os.environ:
                    continue
                os.environ[key] = str(value)
                
        return validated_config

In [ ]:
#| hide
# assume we're only dealing with primitive types for now
example_properties_schema = {
    'title': 'example-properties-schema',
    'description': 'example JSON Schema for demonstration and testing in documentation using nbdev',
    'type': 'object',
    'required': [
        'string_value_with_enum',
        'MY_INTEGER_VALUE',
        'A_NUMERIC_VALUE',
    ],
    'properties': {
        # string
        'SOME_STRING_VALUE_funnyCaSe345': {
            'type': 'string',
        },
        'string_value_with_enum': {
            'description': 'this one is in the <required> list!',
            'type': 'string',
            'enum': [
                'it', 'can', 'only', 'be', 'one', 'of', 'these',
            ],
        },
        '_____A_STRING_VALUE____with_default__': {
            'description': 'values with a default get hydrated using the default if not present in input',
            'type': 'string',
            'default': 'underscores_and spaces',
        },
        
        # integer
        'MY_INTEGER_VALUE': {
            'type': 'integer',
            'description': 'not used for validation, but your benefit',
        },
        
        # number
        'A_NUMERIC_VALUE': {
            'type': 'number',
            'description': 'continuous and real and reasonable',
            'minimum': 22,
            'maximum': 33333.4,
        },
        
        # boolean
        'true_or_false__but_also_nothing': {
            'type': 'boolean',
        }
    },
}

In [ ]:
#| hide
test_fail(ConfigValidator.load_validated_config, args=(example_properties_schema, {}))

In [ ]:
#| hide
test_eq(ConfigValidator.load_validated_config(example_properties_schema, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '1122334',
    'A_NUMERIC_VALUE': '24.89',
}), {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': 1122334,
    'A_NUMERIC_VALUE': 24.89,
    '_____A_STRING_VALUE____with_default__': 'underscores_and spaces',
})

In [ ]:
#| hide
test_eq(ConfigValidator.load_validated_config(example_properties_schema, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '-85',
    'A_NUMERIC_VALUE': '1.23e4',
}), {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': -85,
    'A_NUMERIC_VALUE': 12300.0,
    '_____A_STRING_VALUE____with_default__': 'underscores_and spaces',
})

In [ ]:
#| hide
test_fail(ConfigValidator.load_validated_config,
          'should fail because string_value_with_enum is outside enum',
          args=(example_properties_schema, {
    'string_value_with_enum': 'blah-blah',
    'MY_INTEGER_VALUE': '1122334',
    'A_NUMERIC_VALUE': '24.89',
}))

In [ ]:
#| hide
test_fail(ConfigValidator.load_validated_config,
          'should fail because MY_INTEGER_VALUE is not an integer',
          args=(example_properties_schema, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '5555.999',
    'A_NUMERIC_VALUE': '24.89',
}))

In [ ]:
#| hide
test_fail(ConfigValidator.load_validated_config,
          'should fail because A_NUMERIC_VALUE is not numeric',
          args=(example_properties_schema, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '1122334',
    'A_NUMERIC_VALUE': 'WHAT???',
}))

In [ ]:
#| hide
test_fail(ConfigValidator.load_validated_config,
          'should fail beacuse A_NUMERIC_VALUE is less than the allowed minimum',
          args=(example_properties_schema, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '1122334',
    'A_NUMERIC_VALUE': '13',
}))

In [ ]:
#| hide
from morefs.memory import MemFS

In [ ]:
#| hide
# test ability to override the storage driver (MemFS here)

memfs = MemFS()

memfs.makedirs('extra-long-directory-place', exist_ok=True)
temp_config_validator_json_schema_path = 'extra-long-directory-place/schema.json'
with memfs.open(temp_config_validator_json_schema_path, 'w') as ofile:
    ofile.write(json.dumps(example_properties_schema))
    os.environ['CONFIG_VALIDATOR_JSON_SCHEMA'] = temp_config_validator_json_schema_path

validator = ConfigValidator(storage_driver=memfs)
validated_config = validator.load_config({
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': '-85',
    'A_NUMERIC_VALUE': '1.23e4',
})

test_eq(validated_config, {
    'string_value_with_enum': 'these',
    'MY_INTEGER_VALUE': -85,
    'A_NUMERIC_VALUE': 12300.0,
    '_____A_STRING_VALUE____with_default__': 'underscores_and spaces',
})

# test loading dotenv from an arbitrary file

memfs.makedirs('special-bespoke-location', exist_ok=True)
with memfs.open('special-bespoke-location/my-own.env', 'w') as ofile:
    ofile.write('\n'.join([
        'string_value_with_enum=only',
        'MY_INTEGER_VALUE=9989998',
        'A_NUMERIC_VALUE=1167.89',
    ]))

validated_dotenv = validator.load_dotenv(
    dotenv_path='special-bespoke-location/my-own.env',
    storage_driver=memfs,
)

test_eq(validated_dotenv, {
    'string_value_with_enum': 'only',
    'MY_INTEGER_VALUE': 9989998,
    'A_NUMERIC_VALUE': 1167.89,
    '_____A_STRING_VALUE____with_default__': 'underscores_and spaces',
})

del os.environ['CONFIG_VALIDATOR_JSON_SCHEMA']

In [ ]:
#| hide
# test non-os FS with fallback .env path (=$PWD/.env)

memfs_fallback = MemFS()

with memfs_fallback.open('schema.json', 'w') as ofile:
    ofile.write(json.dumps(example_properties_schema))
    
with memfs_fallback.open('.env', 'w') as ofile:
    ofile.write('\n'.join([
        'string_value_with_enum=only',
        'MY_INTEGER_VALUE=9989998',
        'A_NUMERIC_VALUE=1167.89',
    ]))

OLD_DRIVER = ConfigValidator.DEFAULT_STORAGE_DRIVER
ConfigValidator.DEFAULT_STORAGE_DRIVER = memfs_fallback

validated_config5 = ConfigValidator.load_dotenv(
    json_schema='schema.json',
)
    
ConfigValidator.DEFAULT_STORAGE_DRIVER = OLD_DRIVER

In [ ]:
#| hide
# test using custom json schema

with memfs.open('foo.schema.json', 'w') as ofile:
    ofile.write(json.dumps({
        'type': 'object',
        'properties': {
            'A_NUMERIC_VALUE': { 'type': 'number' },
        }
    }))
validated_dotenv = ConfigValidator.load_dotenv(
    json_schema='foo.schema.json',
    dotenv_path='special-bespoke-location/my-own.env',
    storage_driver=memfs,
)
test_eq(validated_dotenv, {
    'A_NUMERIC_VALUE': 1167.89,
})

test_fail(validator.load_dotenv, kwargs={'dotenv_path': 'non-existent-location-own.env'})

In [ ]:
#| hide
# test data load precedence

with memfs.open('bar.schema.json', 'w') as ofile:
    ofile.write(json.dumps({
        'type': 'object',
        'properties': {
            'A_VALUE_TO_OVERRIDE': { 'type': 'string', 'default': 'change me' },
        }
    }))
    
memfs.makedirs('precedence-test', exist_ok=True)
with memfs.open('precedence-test/.env', 'w') as ofile:
    ofile.write('\n'.join([
        'A_VALUE_TO_OVERRIDE=in dotenv',
    ]))

os.environ.pop('A_VALUE_TO_OVERRIDE', None)
validated_dotenv = ConfigValidator.load_dotenv(
    json_schema='bar.schema.json',
    dotenv_path='precedence-test/.env',
    storage_driver=memfs,
)
test_eq(validated_dotenv, {
    'A_VALUE_TO_OVERRIDE': 'in dotenv',
})

os.environ['A_VALUE_TO_OVERRIDE'] = 'overrode from environ'
test_eq(ConfigValidator.load_dotenv(
    json_schema='bar.schema.json',
    storage_driver=memfs,
), {
    'A_VALUE_TO_OVERRIDE': 'overrode from environ',
})

test_eq(ConfigValidator.load_dotenv(
    json_schema='bar.schema.json',
    dotenv_path='precedence-test/.env',
    storage_driver=memfs,
), {
    'A_VALUE_TO_OVERRIDE': 'overrode from environ',
})

os.environ.pop('A_VALUE_TO_OVERRIDE', None)
test_eq(ConfigValidator.load_dotenv(
    json_schema='bar.schema.json',
    dotenv_path='precedence-test/.env',
    storage_driver=memfs,
), {
    'A_VALUE_TO_OVERRIDE': 'in dotenv',
})

In [ ]:
#| hide
# test loading out of CWD


'''
possibly due to jupyter environment blackmagicfuddery, mocking builtins.open DOES NOT WORK!
take the easy way out and mock an actual file!
'''
import os.path as _p
import tempfile

OLD_DRIVER = ConfigValidator.DEFAULT_STORAGE_DRIVER
ConfigValidator.DEFAULT_STORAGE_DRIVER = None

with tempfile.TemporaryDirectory() as tempdir:

    orig_dir = os.getcwd()
    os.chdir(tempdir)

    dotenv_path = '.flag-env1'
    with open(dotenv_path, 'wt') as f:
        f.write('WINTER=COLD\n')

    json_schema_path = 'flag.schema.json'
    with open(json_schema_path, 'wt') as f:
        schema = {"type": "object", "properties": {"WINTER": {"type": "string"}, "SUMMER": {"type": "string", "default": "HOT"}}}
        json.dump(schema, f)

    dotenv_abspath = _p.join(tempdir, dotenv_path)
    json_schema_abspath = _p.join(tempdir, json_schema_path)

    validated_config = ConfigValidator.load_dotenv(
        json_schema=json_schema_path,
        dotenv_path=dotenv_path
    )
    test_eq(validated_config, {'WINTER': 'COLD', 'SUMMER': 'HOT'})
    os.chdir(orig_dir)

    # test ConfigValidator continues to use the working directory on first invocation
    validated_config2 = ConfigValidator.load_dotenv(
        json_schema=json_schema_path,
        dotenv_path=dotenv_abspath
    )
    test_eq(validated_config, validated_config2)

    validated_config3 = ConfigValidator.load_dotenv(
        json_schema=json_schema_abspath,
        dotenv_path=dotenv_abspath
    )
    test_eq(validated_config, validated_config2)

ConfigValidator.DEFAULT_STORAGE_DRIVER = OLD_DRIVER

In [ ]:
#| hide
# test propagating values into os.environ depending on flag

def test_propagate_values_into_os_environ():
    
    with memfs.open('flag.schema.json', 'w') as ofile:
        ofile.write(json.dumps({
            'type': 'object',
            'properties': {
                'UNINVITED_GUEST': { 'type': 'string', 'default': 'from schema' },
                'convert_me': { 'type': 'integer', 'default': 1 },
            }
        }))
    
    with memfs.open('.flag-env1', 'w') as ofile:
        ofile.write('')
    
    with memfs.open('.flag-env2', 'w') as ofile:
        ofile.write('\n'.join([
            'UNINVITED_GUEST=from dotenv',
        ]))
    
    with memfs.open('.flag-env3', 'w') as ofile:
        ofile.write('\n'.join([
            'UNINVITED_GUEST=I should be ignored!',
        ]))

    mock_env = {"already_here": "no touch me"}

    with patch.dict('os.environ', mock_env):
        
        # don't update os.environ
        validated_config1 = ConfigValidator.load_dotenv(
            json_schema='flag.schema.json',
            dotenv_path='.flag-env1',
            storage_driver=memfs,
        )
        test_eq(os.environ.get("already_here"), "no touch me")
        test_eq(os.environ.get("UNINVITED_GUEST"), None)
        test_eq(validated_config1.get("UNINVITED_GUEST"), "from schema")
        
        # update os.environ, loading from schema
        validated_config2 = ConfigValidator.load_dotenv(
            json_schema='flag.schema.json',
            dotenv_path='.flag-env1',
            storage_driver=memfs,
            override=True
        )
        test_eq(validated_config1, validated_config2)
        test_eq(os.environ.get("UNINVITED_GUEST"), "from schema")
        test_eq(validated_config1.get("convert_me"), 1)
        test_eq(os.environ.get("convert_me"), "1")
        
        os.environ.pop('UNINVITED_GUEST')
        # update os.environ, loading from dotenv
        validated_config3 = ConfigValidator.load_dotenv(
            json_schema='flag.schema.json',
            dotenv_path='.flag-env2',
            storage_driver=memfs,
            override=True
        )
        test_eq(os.environ.get("UNINVITED_GUEST"), "from dotenv")
        
        # os.environ is set; takes precedence
        validated_config3 = ConfigValidator.load_dotenv(
            json_schema='flag.schema.json',
            dotenv_path='.flag-env3',
            storage_driver=memfs,
            override=True
        )
        test_eq(os.environ.get("UNINVITED_GUEST"), "from dotenv")
    
test_propagate_values_into_os_environ()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()